In [1]:
# ============================================================================
# CELL 1: Matplotlib Backend
# ============================================================================
%matplotlib inline


# ============================================================================
# CELL 2: Imports
# ============================================================================
from numpy import sqrt, cos, tan, sin, radians, degrees, pi, arctan2
import matplotlib.patches as mpatches
from scipy.optimize import fsolve
from scipy.integrate import quad
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from ipywidgets import interact, FloatSlider, fixed
import ipywidgets as widgets


# ============================================================================
# CELL 3: Constants and Configuration
# ============================================================================
g = 9.81
rim_width = 1.2192  # 4ft
rim_height = 2.64
cargo_radius = 0.2413/2


# ============================================================================
# CELL 4: Speed Calculation Functions
# ============================================================================
def get_speed_func(startpt, endpt):
    x0, y0 = startpt
    x1, y1 = endpt
    return lambda a: sqrt(0.5*g / (y0-y1 + (x1-x0)*tan(a))) * (x1-x0)/cos(a)

def get_speed_func_squared(startpt, endpt):
    x0, y0 = startpt
    x1, y1 = endpt
    return lambda a: (0.5*g / (y0-y1 + (x1-x0)*tan(a))) * ((x1-x0)/cos(a))**2


# ============================================================================
# CELL 5: Angle-Speed Space Calculation
# ============================================================================
def get_ang_speed_space(xpos, ypos, doShow=False):
    f_far_squared = get_speed_func_squared((xpos, ypos), (rim_width/2, rim_height))
    f_near_squared = get_speed_func_squared((xpos, ypos), (-rim_width/2, rim_height))
    f_squared_diff = lambda a: f_far_squared(a) - f_near_squared(a)
    intersection = fsolve(f_squared_diff, radians(85))[0]

    ang_lower_bound = max(intersection, radians(5))
    ang_upper_bound = radians(85)

    f_far = lambda a: sqrt(f_far_squared(a))
    f_near = lambda a: sqrt(f_near_squared(a))
    f_diff = lambda a: f_far(a) - f_near(a)
    area, error = quad(f_diff, ang_lower_bound, ang_upper_bound)

    angles = np.linspace(degrees(ang_lower_bound), degrees(ang_upper_bound), num=50)
    lower_bound_pts = np.vectorize(f_near)(radians(angles))
    upper_bound_pts = np.vectorize(f_far)(radians(angles))

    if doShow:
        print(f'intersection at angle = {degrees(intersection)} degrees')
        print(f'integrating from {ang_lower_bound} to {ang_upper_bound} radians')
        print(f'{area} area')
        plt.figure()
        plt.fill_between(angles, lower_bound_pts, upper_bound_pts)
        plt.xlabel("angle (degrees)")
        plt.ylabel("speed (m/s)")
        plt.xlim([5, 85])
        plt.ylim([0, 15])

    return area, angles, lower_bound_pts, upper_bound_pts


# ============================================================================
# CELL 6: Flight Physics Model
# ============================================================================
def flight_model(t, s):
    x, vx, y, vy = s
    dx = vx
    dvx = 0
    dy = vy
    dvy = -9.8
    return [dx, dvx, dy, dvy]

def hit_ground(t, s):
    x, vx, y, vy = s
    return y
hit_ground.terminal = True

def hit_rim(t, s):
    x, vx, y, vy = s
    dist_to_rim = min(x - -rim_width/2, -(y - rim_height))
    return dist_to_rim + cargo_radius
hit_rim.terminal = True

def passed_rim(t, s):
    x, vx, y, vy = s
    return x - rim_width/2
passed_rim.terminal = True


# ============================================================================
# CELL 7: Shot Simulation Function
# ============================================================================
def try_shot(s0):
    t_span = (0, 5.0)
    solution = solve_ivp(flight_model, t_span, s0, events=[hit_ground, hit_rim, passed_rim], max_step=0.2)

    result = 0
    if solution.y[0][-1] < -rim_width/2:
        result = -1
    elif solution.y[0][-1] > rim_width/2 - cargo_radius:
        result = 1

    return result, solution.y[0, :], solution.y[2, :]


# ============================================================================
# CELL 8: Generate Area Grid (Pre-compute)
# ============================================================================
x_range = np.arange(-6, -1, 0.1)
y_range = np.arange(0.2, 1.25, 0.1)

area_grid = np.zeros((x_range.size, y_range.size))

for xi in range(x_range.size):
    for yi in range(y_range.size):
        area, angles, lower_bound_pts, upper_bound_pts = get_ang_speed_space(x_range[xi], y_range[yi], doShow=False)
        area_grid[xi][yi] = area * arctan2(rim_width, abs(x_range[xi]))

X, Y = np.meshgrid(x_range, y_range)
mesh_color = area_grid.T

print("Area grid computed!")


# ============================================================================
# CELL 9: Interactive Plotting Function
# ============================================================================
def plot_interactive(x_pos, y_pos, angle_deg, speed):
    """
    x_pos: x position of shooter (meters)
    y_pos: y position of shooter (meters)
    angle_deg: launch angle (degrees)
    speed: launch speed (m/s)
    """
    # Calculate velocity components
    angle_rad = radians(angle_deg)
    vx = speed * cos(angle_rad)
    vy = speed * sin(angle_rad)
    
    shoot_state = [x_pos, vx, y_pos, vy]
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 12))
    fig.suptitle('Projectile Motion Simulator for FRC 2022\nColor shows allowable error in making the shot')
    
    # Plot 1: Position and trajectory
    ax1.scatter(X, Y, c=mesh_color, marker='s', alpha=0.6)
    ax1.set_xlim([-6.2, 1])
    ax1.set_ylim([-0.1, 4])
    ax1.set_xlabel('x position (meters)')
    ax1.set_ylabel('y position (meters)')
    
    # Draw hub/rim
    left, bottom, width, height = (-rim_width/2, 0, rim_width, rim_height)
    ax1.add_patch(mpatches.Rectangle((left, bottom), width, height, fill=False, color="gray", linewidth=2))
    ax1.add_patch(mpatches.Rectangle((-(2.72/2-0.34), 0), 2.72-2*0.34, 0.57, fill=False, color="gray", linewidth=2))
    ax1.add_patch(mpatches.Rectangle((-6, -0.5), 7, 0.5, fill=True, color="gray", linewidth=2))
    ax1.set_aspect('equal', adjustable='box')
    ax1.set_title("Trajectory View")
    
    # Simulate and plot trajectory
    result, traj_x, traj_y = try_shot(shoot_state)
    color_str = 'green' if result == 0 else 'red'
    ax1.plot(traj_x, traj_y, color_str, linewidth=2, label='Success' if result == 0 else 'Miss')
    ax1.scatter(x_pos, y_pos, c='black', s=100, marker='o', zorder=5)
    ax1.legend()
    
    # Plot 2: Angle-Speed space
    area, angles, lower_bound_pts, upper_bound_pts = get_ang_speed_space(x_pos, y_pos, doShow=False)
    ax2.fill_between(angles, lower_bound_pts, upper_bound_pts, color='lightgreen', alpha=0.5, label='Valid shots')
    ax2.set_xlabel("Launch Angle (degrees)")
    ax2.set_ylabel("Launch Speed (m/s)")
    ax2.set_xlim([5, 85])
    ax2.set_ylim([0, 15])
    ax2.set_title("Angle-Speed Space")
    ax2.grid(True, alpha=0.3)
    
    # Plot current shot parameters
    current_speed = sqrt(vx**2 + vy**2)
    ax2.scatter(angle_deg, current_speed, c='red', s=100, marker='x', linewidth=3, zorder=5, label='Current shot')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()


# ============================================================================
# CELL 10: Interactive Controls
# ============================================================================
# Create interactive sliders
interact(plot_interactive,
         x_pos=FloatSlider(min=-6.0, max=-1.0, step=0.1, value=-2.8, description='X Position'),
         y_pos=FloatSlider(min=0.2, max=1.2, step=0.05, value=0.4, description='Y Position'),
         angle_deg=FloatSlider(min=5, max=85, step=1, value=70, description='Angle (°)'),
         speed=FloatSlider(min=1, max=15, step=0.1, value=8.0, description='Speed (m/s)'))

Area grid computed!


interactive(children=(FloatSlider(value=-2.8, description='X Position', max=-1.0, min=-6.0), FloatSlider(value…

<function __main__.plot_interactive(x_pos, y_pos, angle_deg, speed)>